<a href="https://colab.research.google.com/github/jadewisemann/Smart_Farm_Monitoring_System/blob/main/sensor/AI/weed_detection_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
Yolo_Directory = 'weedai_yolo'

%cd /content/drive/MyDrive
%rm -r {Yolo_Directory}

!mkdir /content/drive/MyDrive/{Yolo_Directory}
%ls '/content/drive/MyDrive/'

/content/drive/MyDrive
'Colab Notebooks'/   images/   keypath/   weedai_yolo/   weeddata.zip


In [3]:
%cd /content/drive/MyDrive/{Yolo_Directory}
!git clone https://github.com/ultralytics/yolov5

/content/drive/MyDrive/weedai_yolo
Cloning into 'yolov5'...
remote: Enumerating objects: 15921, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 15921 (delta 17), reused 28 (delta 13), pack-reused 15880
Receiving objects: 100% (15921/15921), 14.66 MiB | 14.50 MiB/s, done.
Resolving deltas: 100% (10916/10916), done.


In [4]:
Yolo_Dataset = 'weeddata'

In [5]:
!mkdir /content/drive/MyDrive/{Yolo_Directory}/yolov5/datasets
%cd '/content/drive/MyDrive/{Yolo_Directory}/yolov5/datasets'

/content/drive/MyDrive/weedai_yolo/yolov5/datasets


In [6]:
!touch data.yaml
!echo -e "path: /content/drive/MyDrive/weedai_yolo/yolov5/datasets\ntrain: images/train\nval: images/val\ntest: images/test\n\nnc: 7\nnames: ['poaceae', 'chenopodium album', 'helianthus annuus','vigna unguiculata', 'erigeron canadensis', 'abutilon theophrasti', 'trifolium incarnatum']"> data.yaml

In [7]:
!unzip -q /content/drive/MyDrive/weeddata.zip

In [8]:
import json
import os
from PIL import Image
import shutil
import json
import os
from PIL import Image

def coco_to_yolo(file, image_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    elif os.listdir(output_dir):
        print("output_dir에 파일이 이미 존재합니다. 실행을 중단합니다.")
        return

    with open(file, 'r') as f:
        data = json.load(f)

    images = {img['id']: img['file_name'] for img in data['images']}
    categories = {cat['id']: cat['name'] for cat in data['categories']}

    annotations = {}
    for ann in data['annotations']:
        image_id = ann['image_id']
        cat_id = ann['category_id']
        bbox = ann['bbox']

        img_file = images[image_id]
        img = Image.open(os.path.join(image_dir, img_file))
        width, height = img.size
        x_center = (bbox[0] + bbox[2] / 2) / width
        y_center = (bbox[1] + bbox[3] / 2) / height
        w = bbox[2] / width
        h = bbox[3] / height

        if image_id not in annotations:
            annotations[image_id] = []
        annotations[image_id].append((cat_id, x_center, y_center, w, h))

    for image_id, objects in annotations.items():
        file_name = images[image_id].replace('.jpg', '.txt').replace('.png', '.txt')
        with open(os.path.join(output_dir, file_name), 'w') as f:
            for obj in objects:
                f.write(f"{obj[0]} {obj[1]} {obj[2]} {obj[3]} {obj[4]}\n")

coco_file = '/content/drive/MyDrive/weedai_yolo/yolov5/datasets/weedcoco.json'
image_dir = '/content/drive/MyDrive/weedai_yolo/yolov5/datasets/images'
output_dir = '/content/drive/MyDrive/weedai_yolo/yolov5/datasets/labels'

coco_to_yolo(coco_file, image_dir, output_dir)

In [9]:
WEED_COCO_LOCATION = f"/content/drive/MyDrive/{Yolo_Directory}/yolov5/datasets"
# #convert the weedcoco file
# convert_weedcoco_json(json_dir=WEED_COCO_LOCATION)

In [10]:
from sklearn.model_selection import train_test_split

# Read images and annotations
images = [os.path.join(f'{WEED_COCO_LOCATION}/images', x) for x in os.listdir(f'{WEED_COCO_LOCATION}/images')]
annotations = [os.path.join(f'{WEED_COCO_LOCATION}/labels', x) for x in os.listdir(f'{WEED_COCO_LOCATION}/labels') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-val-test splits 80-10-10%
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

%cd {WEED_COCO_LOCATION}
!mkdir images/train images/val images/test labels/train labels/val labels/test

/content/drive/MyDrive/weedai_yolo/yolov5/datasets


In [11]:
#Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'labels/train/')
move_files_to_folder(val_annotations, 'labels/val/')
move_files_to_folder(test_annotations, 'labels/test/')

In [12]:
# Check the images have been moved
print(len(os.listdir('images/train')), len(os.listdir('labels/train')))
print(len(os.listdir('images/val')), len(os.listdir('labels/val')))
print(len(os.listdir('images/test')), len(os.listdir('labels/test')))

581 581
73 73
73 73


In [13]:
# import necessary packages
import torch
from IPython.display import Image  # for displaying images
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(0)

# print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [14]:
%cd ..

/content/drive/MyDrive/weedai_yolo/yolov5


In [15]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.3/609.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [16]:
# train YOLOv5m
BATCH = 8
EPOCHS = 30
IMAGE_SIZE = 640 # (should be one of 320, 640, 1280, 1920)
MODEL = 'm' # (should be one of 'n', 's', 'm', 'l', 'x' and must be in lower case)

# this is the name of your run, and how it will be saved
RUN_NAME = f'datasets_TRAIN_B{str(BATCH)}_E{str(EPOCHS)}_SZ{str(IMAGE_SIZE)}_M{MODEL}'

# avoid making any changes to the below, or check the Ultralytics docs for other commands
!python train.py --img 640 --cfg yolov5{MODEL}.yaml --batch 8 --epochs 30 --data datasets/data.yaml --weights yolov5{MODEL}.pt --name {RUN_NAME}

train: weights=yolov5m.pt, cfg=yolov5m.yaml, data=datasets/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=datasets_TRAIN_B8_E30_SZ640_Mm, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-210-gdd10481 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor

In [24]:
!pip install google-cloud-storage
from google.cloud import storage
from google.oauth2 import service_account
from google.colab import drive
from PIL import Image
import os

drive.mount('/content/drive')

KEY_PATH = '/content/drive/MyDrive/keypath/omega-branch-396406-7356fbd5a834.json'
credentials = service_account.Credentials.from_service_account_file(KEY_PATH)

Img_Directory = 'testimages'
!mkdir /content/drive/MyDrive/weedai_yolo/yolov5/datasets/{Img_Directory}

bucket_name = 'smartfarm-image'
source_blob_name = 'd8:3a:dd:27:ec:e0-20230821-111412.jpg'
destination_file_name = '/content/drive/MyDrive/weedai_yolo/yolov5/datasets/testimages/file.jpg'

storage_client = storage.Client(credentials = credentials, project = credentials.project_id)
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(source_blob_name)
blob.download_to_filename(destination_file_name)

image_path = 'testimages/file.jpg'


#for detect
DETECTION_FILES = image_path # e.g. 'test_video.mp4' OR test_image_directory OR test_image.jpg
CONFIDENCE_THRESHOLD = 0.50 # this should be between 0 and 1. It changes the cutoff value for a detection. Lower = more sensitive, higher = less sensitive

!python detect.py --source datasets/{DETECTION_FILES} --weights runs/train/{RUN_NAME}/weights/best.pt --name {RUN_NAME} --img {IMAGE_SIZE} --conf-thres 0.50

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/drive/MyDrive/weedai_yolo/yolov5/datasets/testimages’: File exists
detect: weights=['runs/train/datasets_TRAIN_B8_E30_SZ640_Mm/weights/best.pt'], source=datasets/testimages/file.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=datasets_TRAIN_B8_E30_SZ640_Mm, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-210-gdd10481 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20877180 parameters, 0 gradients, 47.9 GFLOPs
image 1/1 /content/drive/MyDrive